In [1]:
#two model
#one with 5 polarities
#one with 3 polarities

#-------------------------
'''
0 - negative

1 - somewhat negative

2 - neutral

3 - somewhat positive

4 - positive

'''


#----------------------------
'''
0 - negative
1 - neutral
2 - positive

'''

'\n0 - negative\n1 - neutral\n2 - positive\n\n'

In [1]:
import numpy as np 
import pandas as pd 
import os
from tqdm import tqdm
#importing the NLP Libraries
from bs4 import BeautifulSoup
import re
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
from keras.utils import to_categorical
import random
from tensorflow import set_random_seed
from keras.optimizers import Adam
from keras.layers import Dense,Dropout,Embedding,LSTM
from keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer
from keras.losses import categorical_crossentropy
from keras.models import Sequential
from keras.models import load_model

set_random_seed(123)



Using TensorFlow backend.


In [0]:
train= pd.read_csv(r"train.tsv", sep="\t")

In [8]:
def clean_sentences(df):
    reviews = []
    for sent in tqdm(df['Phrase']):
        review_text = BeautifulSoup(sent).get_text()
        review_text = re.sub("[^a-zA-Z]"," ", review_text)
        words = word_tokenize(review_text.lower())
        lemma_words = [lemmatizer.lemmatize(i) for i in words]
        reviews.append(lemma_words)
    return(reviews)
train_sentences = clean_sentences(train)
target=train.Sentiment.values
y_target=to_categorical(target)
num_classes=y_target.shape[1]
X_train,X_val,y_train,y_val=train_test_split(train_sentences,y_target,test_size=0.2,stratify=y_target)
#It is needed for initializing tokenizer of keras and subsequent padding

unique_words = set()
len_max = 0

for sent in tqdm(X_train):
    unique_words.update(sent)
    if(len_max<len(sent)):
        len_max = len(sent)
tokenizer = Tokenizer(num_words=len(list(unique_words)))
tokenizer.fit_on_texts(list(X_train))
X_train = tokenizer.texts_to_sequences(X_train)
X_train = sequence.pad_sequences(X_train, maxlen=len_max)

  0%|          | 1/156060 [00:01<64:45:18,  1.49s/it]/usr/local/lib/python3.6/dist-packages/bs4/__init__.py:273: UserWarning: "b'.'" looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  ' Beautiful Soup.' % markup)
100%|██████████| 124848/124848 [00:00<00:00, 699574.59it/s]


In [137]:
def clean_sentences(df):
    df=np.array([df])
    reviews = []
    for sent in tqdm(df):
        #review_text = BeautifulSoup(sent).get_text()
        review_text=str(sent)
        review_text = re.sub("[^a-zA-Z]"," ", review_text)
        words = word_tokenize(review_text.lower())
        lemma_words = [lemmatizer.lemmatize(i) for i in words]
        reviews.append(lemma_words)
    return(reviews)
Y_crit_clean = clean_sentences(Y_criticker)
Y_crit_clean = tokenizer.texts_to_sequences(Y_crit_clean)
Y_sentiment = sequence.pad_sequences(Y_crit_clean, maxlen=len_max)



  0%|          | 0/1 [00:00<?, ?it/s]

100%|██████████| 1/1 [00:00<00:00, 561.34it/s]

In [0]:
model1 = load_model('my_model_with_3_sentiments.h5')
model2 = load_model('my_model_with_5_sentiments.h5')

In [110]:
model1.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (None, 48, 300)           4120500   
_________________________________________________________________
lstm_9 (LSTM)                (None, 48, 128)           219648    
_________________________________________________________________
lstm_10 (LSTM)               (None, 64)                49408     
_________________________________________________________________
dense_9 (Dense)              (None, 100)               6500      
_________________________________________________________________
dropout_5 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_10 (Dense)             (None, 3)                 303       
Total params: 4,396,359
Trainable params: 4,396,359
Non-trainable params: 0
____________________________________________

In [141]:
model2.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 48, 300)           4120500   
_________________________________________________________________
lstm_1 (LSTM)                (None, 48, 128)           219648    
_________________________________________________________________
lstm_2 (LSTM)                (None, 64)                49408     
_________________________________________________________________
dense_1 (Dense)              (None, 100)               6500      
_________________________________________________________________
dropout_1 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 5)                 505       
Total params: 4,396,561
Trainable params: 4,396,561
Non-trainable params: 0
____________________________________________

In [0]:
def predict(y):
  y = clean_sentences(y)
  y = tokenizer.texts_to_sequences(y)
  y = sequence.pad_sequences(y, maxlen=len_max)
  Sentiment_crit1=model1.predict_classes(y)
  Sentiment_crit2=model2.predict_classes(y)
  print("\n")
  print("Model1 with 3 Polarity class output :",Sentiment_crit1[0])
  print("Model1 with 5 Polarity class output :",Sentiment_crit2[0])
  
  


In [0]:
test=pd.read_csv("train.tsv", sep="\t")

'to avoid'

In [166]:
predict(test['Phrase'][100])




  0%|          | 0/1 [00:00<?, ?it/s]

100%|██████████| 1/1 [00:00<00:00, 614.28it/s]



Model1 with 3 Polarity class output : 2
Model1 with 5 Polarity class output : 2
